In [2]:
import json
import requests

# Ryan code starts

#### Make API call for historical dataset and store into local json file

In [3]:
url = "https://api.covid19api.com/all"
response = requests.get(url)
with open("static/data/covid_all.json", 'w') as jsonfile:
    json.dump(response.json(), jsonfile)

In [4]:
with open("static/data/covid_all.json") as f:
    covid = json.load(f)

In [5]:
# code to store data in MongoDB
import pymongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.covid_db
db.historical.drop()

db.historical.insert_many(covid)

In [6]:
confirmed = []
deaths = []
recovered = []

#### Separate historical data by status

In [5]:
for i in range(len(covid)):
    data = covid[i]
    if data['Status'] == 'confirmed':
        confirmed.append(data)
    elif data['Status'] == 'deaths':
        deaths.append(data)
    elif data['Status'] == 'recovered':
        recovered.append(data)

In [7]:
def get_date_list(list):
    dates = []
    for i in range(len(list)):
        if list[i]['Date'] not in dates:
            dates.append(list[i]['Date'])
    return sorted(dates)

In [8]:
dates_confirmed = []
dates_deaths = []
dates_recovered = []
dates_confirmed = get_date_list(confirmed)
dates_deaths = get_date_list(deaths)
dates_recovered = get_date_list(recovered)

In [9]:
covid_clean = {
    'confirmed': {},
    'deaths': {},
    'recovered': {}    
}

In [10]:
def get_date_data(dates, data_list, status):
    for i in range(len(dates)):
        current = []
        for j in range(len(data_list)):
            if dates[i] == data_list[j]["Date"]:
                current.append(data_list[j])
        covid_clean[status][dates[i][0:10]] = current
    return covid_clean

def get_date_data_nostatus(dates, data_list):
    data_nostatus = dict()
    for i in range(len(dates)):
        current = []
        for j in range(len(data_list)):
            if dates[i] == data_list[j]["Date"]:
                current.append(data_list[j])
        data_nostatus[dates[i][0:10]] = current
    return data_nostatus

In [11]:
covid_clean = get_date_data(dates_confirmed, confirmed, 'confirmed')
covid_clean = get_date_data(dates_deaths, deaths, 'deaths')
covid_clean = get_date_data(dates_recovered, recovered, 'recovered')

In [47]:
# covid_clean
# for i in range(len(dates_deaths)):
#     print(len(covid_clean['deaths'][dates_deaths[i]]))

In [51]:
# import pymongo
# conn = 'mongodb://localhost:27017'
# client = pymongo.MongoClient(conn)
# db = client.covid_db
# db.covid_clean.insert_one(covid_clean)  ## this is not working due to oversize

# db.covid_confirmed.insert_one(covid_clean['confirmed'])
# db.covid_deaths.insert_one(covid_clean['deaths'])
# db.covid_recovered.insert_one(covid_clean['recovered'])

In [12]:
url_confirmed = "https://api.covid19api.com/dayone/country/us/status/confirmed"
url_deaths = "https://api.covid19api.com/dayone/country/us/status/deaths"
url_recovered = "https://api.covid19api.com/dayone/country/us/status/recovered"

In [13]:
response_confirmed = requests.get(url_confirmed).json()
dates_us_confirmed = get_date_list(response_confirmed)
us_confirmed_by_dates = get_date_data_nostatus(dates_us_confirmed, response_confirmed)

with open("static/data/covid_us_confirmed.json", 'w') as jsonfile:
    json.dump(us_confirmed_by_dates, jsonfile)

In [14]:
date_list = list(us_confirmed_by_dates.keys())
adate = date_list

In [15]:
response_deaths = requests.get(url_deaths).json()
dates_us_deaths = get_date_list(response_deaths)
us_deaths_by_dates = get_date_data_nostatus(dates_us_deaths, response_deaths)

with open("static/data/covid_us_deaths.json", 'w') as jsonfile:
    json.dump(us_deaths_by_dates, jsonfile)

In [16]:
response_recovered = requests.get(url_recovered).json()
dates_us_recovered = get_date_list(response_recovered)
us_recovered_by_dates = get_date_data_nostatus(dates_us_recovered, response_recovered)

with open("static/data/covid_us_recovered.json", 'w') as jsonfile:
    json.dump(us_recovered_by_dates, jsonfile)

In [15]:
# dates_us_confirmed = get_date_list(covid_confirmed)
# dates_us_deaths = get_date_list(covid_deaths)
# dates_us_recovered = get_date_list(covid_recovered)

In [17]:
import pymongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.covid_db
# db.covid_clean.insert_one(covid_clean)  ## this is not working due to oversize
db.us_only_confirmed.drop()
db.us_only_confirmed.insert_one(us_confirmed_by_dates)
db.us_only_deaths.drop()
db.us_only_deaths.insert_one(us_deaths_by_dates)
db.us_only_recovered.drop()
db.us_only_recovered.insert_one(us_recovered_by_dates)

# Ryan code ends

# Kana code starts

# Kana code ends

# Connor code starts

# Connor code ends

# Nabeel code starts

# Nabeel code ends